In [38]:
pip install gurobipy

In [40]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from math import sqrt
from google.colab import drive
drive.mount("/content/drive")
from google.colab import files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Data Loading**

In [41]:
def check_dataframe(data):
    datatype = pd.DataFrame(data.dtypes,columns=["Data Type"])
    datatype["Missing Values"]=data.isnull().sum()
    datatype["% Missing Values"]=data.isnull().sum()/len(data)*100
    print(datatype.sort_values(by="% Missing Values", ascending=False).style.background_gradient(cmap='Set3',axis=0).format({'% Missing Values': '{:.2f}%'}))
    return datatype.sort_values(by="% Missing Values", ascending=False).style.background_gradient(cmap='Set3',axis=0).format({'% Missing Values': '{:.2f}%'})

In [42]:
fcs_df = pd.read_excel("/content/drive/MyDrive/Năm III/SCM/ROUND 2/data/Fulfillment centers.xlsx")
fcs_df.head()

,Location,Region,Subregion
0,TPHCM 1,South,sr1
1,TPHCM 2,South,sr1
2,TPHCM 3,South,sr1
3,TPHCM 4,South,sr1
4,Ha Noi 1,North,sr4


In [43]:
check_dataframe(fcs_df)

,Data Type,Missing Values,% Missing Values
Location,object,0,0.00%
Region,object,0,0.00%
Subregion,object,0,0.00%


In [44]:
subregions = []
for i in range(6):
  subregions.append(f"sr{i}")

In [45]:
cus_df = pd.read_excel("/content/drive/MyDrive/Năm III/SCM/ROUND 2/data/Customers.xlsx")
cus_df.head()

,Customer Group,Region,Demand 2024
0,TPHCM 1,South,130372.5
1,TPHCM 2,South,130372.5
2,TPHCM 3,South,130372.5
3,TPHCM 4,South,130372.5
4,Ha Noi 1,North,138026.5


In [46]:
check_dataframe(cus_df)

,Data Type,Missing Values,% Missing Values
Customer Group,object,0,0.00%
Region,object,0,0.00%
Demand 2024,float64,0,0.00%


In [47]:
weight_list = []
for i in range(23):
  weight_list.append(f'w{i}')
std_demand_df = pd.read_excel("/content/drive/MyDrive/Năm III/SCM/ROUND 2/data/Standard & order distribution.xlsx", sheet_name = 'Standard')
std_demand_df.head()

,Encoded Weight,Weight/Zone,% Distribution rate
0,w0,0.25,0.335788
1,w1,0.5,0.345239
2,w2,1,0.191355
3,w3,1.5,0.067255
4,w4,2,0.017472


In [48]:
check_dataframe(std_demand_df)

,Data Type,Missing Values,% Missing Values
Encoded Weight,object,0,0.00%
Weight/Zone,object,0,0.00%
% Distribution rate,float64,0,0.00%


In [49]:
exp_demand_df = pd.read_excel("/content/drive/MyDrive/Năm III/SCM/ROUND 2/data/Standard & order distribution.xlsx", sheet_name = 'Express')
exp_demand_df.head()

,Encoded Weight,Weight/Zone,% Distribution rate
0,w0,0.25,0.000000
1,w1,0.5,0.000000
2,w2,1,0.000000
3,w3,1.5,0.229185
4,w4,2,0.066710


In [50]:
check_dataframe(exp_demand_df)

,Data Type,Missing Values,% Missing Values
Encoded Weight,object,0,0.00%
Weight/Zone,object,0,0.00%
% Distribution rate,float64,0,0.00%


In [51]:
std_rate_df = pd.read_excel("/content/drive/MyDrive/Năm III/SCM/ROUND 2/data/StdRate.xlsx", sheet_name='Sheet1')
std_rate_df.head()

,Location,Intra city_w0,Intra city_w1,Intra city_w2,Intra city_w3,Intra city_w4,Intra city_w5,Intra city_w6,Intra city_w7,Intra city_w8,...,North-South_w13,North-South_w14,North-South_w15,North-South_w16,North-South_w17,North-South_w18,North-South_w19,North-South_w20,North-South_w21,North-South_w22
0,TPHCM 1,8689.36,10733.92,13085.16,16560.91,18912.15,20445.56,22490.12,24534.68,26579.24,...,155897.42,166631.34,177365.26,188099.18,198833.10,209567.02,220300.94,231034.86,1089748.46,2163140.46
1,TPHCM 2,8689.36,10733.92,13085.16,16560.91,18912.15,20445.56,22490.12,24534.68,26579.24,...,155897.42,166631.34,177365.26,188099.18,198833.10,209567.02,220300.94,231034.86,1089748.46,2163140.46
2,TPHCM 3,8689.36,10733.92,13085.16,16560.91,18912.15,20445.56,22490.12,24534.68,26579.24,...,155897.42,166631.34,177365.26,188099.18,198833.10,209567.02,220300.94,231034.86,1089748.46,2163140.46
3,TPHCM 4,8689.36,10733.92,13085.16,16560.91,18912.15,20445.56,22490.12,24534.68,26579.24,...,155897.42,166631.34,177365.26,188099.18,198833.10,209567.02,220300.94,231034.86,1089748.46,2163140.46
4,Ha Noi 1,8932.99,11034.87,13452.04,17025.23,19442.40,21018.81,23120.69,25222.57,27324.45,...,160268.37,171303.24,182338.11,193372.98,204407.85,215442.72,226477.59,237512.46,1120302.06,2223789.06


In [52]:
check_dataframe(std_rate_df)

,Data Type,Missing Values,% Missing Values
Location,object,0,0.00%
HN-HCM_w2,float64,0,0.00%
HNHCM-DN_w19,float64,0,0.00%
HNHCM-DN_w20,float64,0,0.00%
HNHCM-DN_w21,float64,0,0.00%
HNHCM-DN_w22,float64,0,0.00%
HN-HCM_w0,float64,0,0.00%
HN-HCM_w1,float64,0,0.00%
HN-HCM_w3,float64,0,0.00%
HN-HCM_w11,float64,0,0.00%


In [53]:
exp_rate_df = pd.read_excel("/content/drive/MyDrive/Năm III/SCM/ROUND 2/data/ExpRate.xlsx", sheet_name='Sheet1')
exp_rate_df.head()

,Location,Intra city_w0,Intra city_w1,Intra city_w2,Intra city_w3,Intra city_w4,Intra city_w5,Intra city_w6,Intra city_w7,Intra city_w8,...,North-South_w13,North-South_w14,North-South_w15,North-South_w16,North-South_w17,North-South_w18,North-South_w19,North-South_w20,North-South_w21,North-South_w22
0,TPHCM 1,30000,30000,30000,30000,30000,34000,34000,38000,38000,...,80000,80000,90000,90000,100000,100000,110000,110000,510000,1010000
1,TPHCM 2,30000,30000,30000,30000,30000,34000,34000,38000,38000,...,80000,80000,90000,90000,100000,100000,110000,110000,510000,1010000
2,TPHCM 3,30000,30000,30000,30000,30000,34000,34000,38000,38000,...,80000,80000,90000,90000,100000,100000,110000,110000,510000,1010000
3,TPHCM 4,30000,30000,30000,30000,30000,34000,34000,38000,38000,...,80000,80000,90000,90000,100000,100000,110000,110000,510000,1010000
4,Ha Noi 1,30000,30000,30000,30000,30000,34000,34000,38000,38000,...,80000,80000,90000,90000,100000,100000,110000,110000,510000,1010000


In [54]:
check_dataframe(exp_rate_df)

,Data Type,Missing Values,% Missing Values
Location,object,0,0.00%
HN-HCM_w2,int64,0,0.00%
HNHCM-DN_w19,int64,0,0.00%
HNHCM-DN_w20,int64,0,0.00%
HNHCM-DN_w21,int64,0,0.00%
HNHCM-DN_w22,int64,0,0.00%
HN-HCM_w0,int64,0,0.00%
HN-HCM_w1,int64,0,0.00%
HN-HCM_w3,int64,0,0.00%
HN-HCM_w11,int64,0,0.00%


## **Mathematical model**

With $I$ as the collective of n potetial facilities, $J$ as the collective of n customers. The theoretical mathematical model for the facility location problem with single sourcing is described as below [(Holmberg, Rönnqvist & Yuan, 1999)](https://doi.org/10.1016/S0377-2217(98)00008-3):

Parameters:

$f_i$ is the fixed cost of using/opening facility i.

$c_{ij}$ is the variable cost of assigning customer j to facility i.

$b_i$ is the capacity of facility i.

$d_j$ is the demand of customer j.

Decision variables:

$y_i=1$ if facility i opens, 0 otherwise.

$x_{ij}=1$ if facility i serves customer j, 0 otherwise.

The objective function is to minimize the cost of assigning customers to open facilities and the cost of establishing such facilities.

Objective function:

$v^* = \sum_{i=1}^{n} y_i\cdot f_i + \sum_{i=1}^{n} \sum_{j=1}^{m} c_{ij}\cdot x_{ij}$

Constraints:

(1) Customer demand served by a certain facility does not exceed its capacity

$\sum_{j=1}^{m}d_j\cdot x_{ij} \leq b_i\cdot y_i$

(2) A customer is served by a single source

$\sum_{i=1}^{n}x_{ij} = 1$

Based on the theoretical model, I added three constraints:

(3) Only 10 fulfillment centers will be opened at max

(4) Each subregion has at least 1 fulfillment center

With $ I $ as the collective of n potetial locations for fulfillment centers, $ J $ as the collective of n customers groups, $ R $ is the collective of region, $ IR_r $ is the collective of potential location in region $ r$, $ T $ is the collective of subregions, and $ SR_t $ is the collective of potential locations for fulfillment centers in subregion $ t$. The modified mathematical model is described as below:

Parameters:

$f_i$ is the cost of using facility i in one year.

$c_{ij}$ is the shipping cost from fulfillment center i to customer j per one unit.

$l_{ij}$ is the lead time from fulfillment center i to customer j per one unit.

$d_j$ is the demand of customer j in one year.

$b_i$ is the capacity of fulfillment center i in one year.

Decision variables:

$y_i=1$ if fulfillment center i opens, 0 otherwise.

$x_{ij}=1$ if fulfillment center i serves customer (group) j, 0 otherwise.

Considering the long-term effect, I do not include the fixed cost of opening the fulfillment center into the model. The objective function is to minimize the total effect of operational cost & shipping cost to serve all the demand in one year.

Objective function:

$v^* = \sum_{i=1}^{n} y_i\cdot f_i + \sum_{i=1}^{n} \sum_{j=1}^{m} d_j\cdot c_{ij}\cdot x_{ij}$

Constraints:

(1) Customer demand served by a certain facility does not exceed its capacity

$\sum_{j=1}^{m}d_j\cdot x_{ij} \leq b_i\cdot y_i \text{ } \forall i \in I$

(2) A customer is served by a single source

$\sum_{i=1}^{n}x_{ij} = 1 \text{ } \forall j \in J$

(3) Only 10 fulfillment centers will be opened at max

$\sum_{i=1}^{n} y_i \leq 10$

(4) Each subregion has at least 1 fulfillment center

$\sum_{i \in SR_t} y_i \geq 1 \text{ } \forall t \in T$

## **Input Preparation**

In [55]:
# Generate fulfillment centers' capacity dictionary
def fcs_capacity_generate(fcs, fixed_cap):
  result = {}
  for fc in fcs:
    result[fc] = fixed_cap
  return result

# Generate customers' demand dictionary
def customer_demand_generate(customers, cus_df):
  result = {}
  for customer in customers:
    result[customer] = cus_df[cus_df['Customer Group'] == customer]['Demand 2024'].values[0]
  return result

# Generate fulfillment centers' operational cost
def fcs_operational_cost_generate(fcs, annual_operational_cost):
  result = {}
  for fc in fcs:
    result[fc] = annual_operational_cost
  return result

# Retrieve shipping type
def shipping_type(fc, customer, fcs_df, cus_df):
  # HCM - HN special cases
  #print('Start comparing HCM & HN')
  if (customer[:-2]=='TPHCM') or (customer[:-2]=='Ha Noi'):
    # If fc and customer are both HCM or are both HN
    if (fc[:-2] == customer[:-2]):
      return "Intra city"
    if (fc[:-2] == "Ha Noi") or (fc[:-2] == "TPHCM"):
      return "HN-HCM"
    if (fc == "Da Nang"):
      return "HNHCM-DN"
  # Da Nang special cases
  #print('Start Da Nang')
  if (customer == 'Da Nang'):
    if (fc[:-2] == 'Ha Noi') or (fc[:-2] == 'TPHCM'):
      return "HNHCM-DN"
  # Other cases
  #print('Start Intra City')
  if (customer == fc):
      return "Intra city"
  #print('Start Intra region')
  if (fcs_df[fcs_df['Location'] == fc]['Region'].values[0] == cus_df[cus_df['Customer Group'] == customer]['Region'].values[0]):
      return "Intra region"
  #print('Start North-South')
  if (fcs_df[fcs_df['Location'] == fc]['Region'].values[0] == 'North') and (cus_df[cus_df['Customer Group'] == customer]['Region'].values[0] == 'South'):
      return "North-South"
  if (fcs_df[fcs_df['Location'] == fc]['Region'].values[0] == 'South') and (cus_df[cus_df['Customer Group'] == customer]['Region'].values[0] == 'North'):
      return "North-South"
  #print('Start NorthSouth-Mid')
  if (fcs_df[fcs_df['Location'] == fc]['Region'].values[0] == 'North') and (cus_df[cus_df['Customer Group'] == customer]['Region'].values[0] == 'Central'):
      return "NorthSouth-Mid"
  if (fcs_df[fcs_df['Location'] == fc]['Region'].values[0] == 'South') and (cus_df[cus_df['Customer Group'] == customer]['Region'].values[0] == 'Central'):
      return "NorthSouth-Mid"
  if (fcs_df[fcs_df['Location'] == fc]['Region'].values[0] == 'Central') and (cus_df[cus_df['Customer Group'] == customer]['Region'].values[0] == 'North'):
      return "NorthSouth-Mid"
  if (fcs_df[fcs_df['Location'] == fc]['Region'].values[0] == 'Central') and (cus_df[cus_df['Customer Group'] == customer]['Region'].values[0] == 'South'):
      return "NorthSouth-Mid"

# Retrieve weight distribution
def weight_type(type, distribution_df):
  return distribution_df[distribution_df['Encoded Weight'] == type]['% Distribution rate'].values[0]

# Retrieve shipping cost
# 80% demand is for standard, 20% demand is for express
def total_shipping_cost_for_one_cusgroup(fc, customer, fcs_df, cus_df, weight_list, std_demand_df, exp_demand_df, std_rate_df, exp_rate_df):
  factor_std = 0
  factor_exp = 0
  result = 0
  demand = cus_df[cus_df['Customer Group'] == customer]['Demand 2024'].values[0]
  for weight in weight_list:
    factor_std = factor_std + weight_type(weight, std_demand_df) * std_rate_df[std_rate_df['Location']==customer][shipping_type(fc, customer, fcs_df, cus_df)+"_"+weight].values[0]
  result = demand*0.8*factor_std
  for weight in weight_list:
    factor_exp = factor_exp + weight_type(weight, exp_demand_df) * exp_rate_df[exp_rate_df['Location']==customer][shipping_type(fc, customer, fcs_df, cus_df)+"_"+weight].values[0]
  result = result + demand*0.2*factor_exp
  return result

def average_shipping_cost_on_one_unit(fc, customer, fcs_df, cus_df, weight_list, std_demand_df, exp_demand_df, std_rate_df, exp_rate_df):
  factor_std = 0
  factor_exp = 0
  result = 0
  #print("Demand:", demand)
  for weight in weight_list:
    #print("Weight:", weight)
    #print('Rate:', std_rate_df[std_rate_df['Location']==customer][shipping_type(fc, customer, fcs_df, cus_df)+"_"+weight])
    factor_std = factor_std + weight_type(weight, std_demand_df) * std_rate_df[std_rate_df['Location']==customer][shipping_type(fc, customer, fcs_df, cus_df)+"_"+weight].values[0]
    #print("Factor standard:", factor_std)
  result = 0.8*factor_std
  for weight in weight_list:
    factor_exp = factor_exp + weight_type(weight, exp_demand_df) * exp_rate_df[exp_rate_df['Location']==customer][shipping_type(fc, customer, fcs_df, cus_df)+"_"+weight].values[0]
  result = result+0.2*factor_exp
  return result

# Generate shipping cost dictionary
def shipping_cost_generate(fcs, customers, fcs_df, cus_df, weight_list, std_demand_df, exp_demand_df, std_rate_df, exp_rate_df):
  result = {}
  for fc in fcs:
    for customer in customers:
      #print("FC:", fc)
      #print("Customer:", customer)
      result[(fc, customer)] = average_shipping_cost_on_one_unit(fc, customer, fcs_df, cus_df, weight_list, std_demand_df, exp_demand_df, std_rate_df, exp_rate_df)
  return result

# Generate subregion dictionary
def subregion_generate(fcs_df, subregions):
  result = {}
  for sub in subregions:
    result[sub] = list(fcs_df[fcs_df['Subregion'] == sub]['Location'].values)
  return result


# Retrieve average shipping cost for result
def average_shipping_result(assignFC, customers, customer_demand, fcs_df, cus_df, weight_list, std_demand_df, exp_demand_df, std_rate_df, exp_rate_df):
  i = 0
  total_demand = 0
  result = 0
  while i < len(customers):
    ship_cost = total_shipping_cost_for_one_cusgroup(assignFC[i], customers[i], fcs_df, cus_df, weight_list, std_demand_df, exp_demand_df, std_rate_df, exp_rate_df)
    total_demand = total_demand + customer_demand[customers[i]]
    result = result + ship_cost
    i = i + 1
  result = result/total_demand
  return result

# Retrieve exact total operational and shipping cost
def total_operation_shipping_cost(assignFC, customers, chosen_FCs_df, customer_demand, fcs_df, cus_df, weight_list, std_demand_df, exp_demand_df, std_rate_df, exp_rate_df):
  i = 0
  result = 0
  # Shipping cost
  while i < len(customers):
    ship_cost = total_shipping_cost_for_one_cusgroup(assignFC[i], customers[i], fcs_df, cus_df, weight_list, std_demand_df, exp_demand_df, std_rate_df, exp_rate_df)
    result = result + ship_cost
    i = i + 1
  # Operational cost, demand assignd > 80000 -> 762000000, demand assigned <= 80000 -> 450000000
  result = result + sum(chosen_FCs_df[chosen_FCs_df["Demand assigned"] > 80000]["Open"]*762000000) + sum(chosen_FCs_df[chosen_FCs_df["Demand assigned"] <= 80000]["Open"]*450000000)
  return result

In [56]:
# Retrieve input
fcs = list(fcs_df['Location'])
customers = list(cus_df['Customer Group'])
fcs_capacity = fcs_capacity_generate(fcs, 250000)
customer_demand = customer_demand_generate(customers, cus_df)
fcs_operational_cost = fcs_operational_cost_generate(fcs, 606000000)
shipping_cost = shipping_cost_generate(fcs, customers, fcs_df, cus_df, weight_list, std_demand_df, exp_demand_df, std_rate_df, exp_rate_df)

## **Optimal Fulfillment Centers With Subregion Constraint (4)**

In [57]:
subregion_dict = subregion_generate(fcs_df, subregions)

In [58]:
# Create a new Gurobi model
model = gp.Model()

# Decision Variables
yi = model.addVars(fcs, vtype=GRB.BINARY, name="yi")  # Fulfillment center opens
xij = model.addVars(fcs, customers, vtype=GRB.BINARY, name="xij")  # Fulfillment center serves customer group j

# Objective Function
model.setObjective(
    gp.quicksum(yi[i] * fcs_operational_cost[i] for i in fcs) +
    gp.quicksum(customer_demand[j] * shipping_cost[i, j] * xij[i, j] for i in fcs for j in customers),
    sense=GRB.MINIMIZE
)

# Constraints
for i in fcs:
    model.addConstr(
        gp.quicksum(customer_demand[j] * xij[i, j] for j in customers) <= fcs_capacity[i] * yi[i],
        name=f"capacity_constraint_{i}"
    )

for j in customers:
    model.addConstr(
        gp.quicksum(xij[i, j] for i in fcs) == 1,
        name=f"single_source_constraint_{j}"
    )

model.addConstr(
    gp.quicksum(yi[i] for i in fcs) <= 10,
    name="max_fulfillments_constraint"
)

for t in subregions:
    model.addConstr(
        gp.quicksum(yi[i] for i in subregion_dict[t]) >= 1,
        name=f"subregion_constraint_{t}"
    )

# Solve the optimization problem
model.optimize()

# Print solution
if model.status == GRB.OPTIMAL:
    openFC = []
    demandAssigned = []
    print("Optimal solution found.")
    for i in fcs:
      demand = 0
      if (yi[i].x == 1):
        openFC.append(1)
        print(f"Open Fulfillment Center FC {i}")
        for j in customers:
          if xij[i, j].x == 1:
            demand = demand + customer_demand[j]
      else:
        openFC.append(0)
      demandAssigned.append(demand)
    assignFC = []
    for j in customers:
      for i in fcs:
        if (xij[i, j].x == 1):
          assignFC.append(i)
          print(f"Customer Cus {j} is assigned to fulfillment centers {i}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 100 rows, 1953 columns and 3937 nonzeros
Model fingerprint: 0x46463344
Variable types: 0 continuous, 1953 integer (1953 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [1e+07, 1e+10]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 5.967656e+10
Presolve removed 1 rows and 1 columns
Presolve time: 0.03s
Presolved: 99 rows, 1952 columns, 3934 nonzeros
Variable types: 0 continuous, 1952 integer (1952 binary)
Found heuristic solution: objective 5.821958e+10

Root relaxation: objective 4.101206e+10, 112 iterations, 0.00 seconds (0.00 work uni

In [59]:
chosen_FCs_df = pd.DataFrame({"FCs": fcs, "Open": openFC, "Demand assigned": demandAssigned})
chosen_FCs_df.to_excel('FCs With Subregion Constraint.xlsx', index=False)
files.download('FCs With Subregion Constraint.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [60]:
customer_assigned_df = pd.DataFrame({"Customer Groups": customers, "Assigned to": assignFC})
customer_assigned_df.to_excel('Customer Assigned With Subregion Constraint.xlsx', index=False)
files.download('Customer Assigned With Subregion Constraint.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>